In [1]:
import numpy as np
from tensorflow import keras
from sklearn.metrics import f1_score
from models.base_model.layers.activation_layer import ReLU, Softmax
from models.base_model.layers.dense_layer import DenseLayer
from models.base_model.layers.conv_2d_layer import Conv2DLayer
from models.base_model.layers.max_pooling_2d_layer import MaxPooling2DLayer
from models.base_model.layers.average_pooling_2d_layer import AveragePooling2DLayer
from models.base_model.layers.flatten_layer import FlattenLayer
from models.cnn.cnn import CNN

2025-05-28 15:30:28.740960: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 15:30:28.758901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748421028.776826   45142 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748421028.781646   45142 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748421028.796537   45142 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Load CIFAR-10
(x_train_full, y_train_full), (x_test, y_test) = (
    keras.datasets.cifar10.load_data()
)

# Normalize pixel values
x_train_full = x_train_full.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Create train/validation split (40k train, 10k validation)
split_idx = 40000

x_train = x_train_full[:split_idx]
y_train = y_train_full[:split_idx].flatten()
x_val = x_train_full[split_idx:]
y_val = y_train_full[split_idx:].flatten()
x_test = x_test
y_test = y_test.flatten()

print(f"Training set: {x_train.shape}")
print(f"Validation set: {x_val.shape}")
print(f"Test set: {x_test.shape}")

Training set: (40000, 32, 32, 3)
Validation set: (10000, 32, 32, 3)
Test set: (10000, 32, 32, 3)


In [4]:
def create_keras_model(
    filters_list=[32, 64, 128], kernel_sizes=[3, 3, 3], pooling_type="max"
):
    model = keras.Sequential()

    # Add convolutional layers
    for i, (filters, kernel_size) in enumerate(zip(filters_list, kernel_sizes)):
        if i == 0:
            model.add(
                keras.layers.Conv2D(
                    filters, kernel_size, activation="relu", input_shape=(32, 32, 3), padding="same"
                )
            )
        else:
            model.add(keras.layers.Conv2D(filters, kernel_size, activation="relu", padding="same"))

        # Add pooling layer
        if pooling_type == "max":
            model.add(keras.layers.MaxPooling2D((2, 2)))
        else:
            model.add(keras.layers.AveragePooling2D((2, 2)))

    # Add dense layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))

    return model

def train_keras_model(model, epochs=10):
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    history = model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=epochs,
        validation_data=(x_val, y_val),
        verbose=1,
    )

    return history

In [5]:
def create_scratch_model(
    filters_list=[32, 64, 128], kernel_sizes=[3, 3, 3], pooling_type="max"
):
    cnn = CNN(input_shape=(32, 32, 3), num_classes=10)

    # Add convolutional layers
    for _, (filters, kernel_size) in enumerate(zip(filters_list, kernel_sizes)):
        cnn.add(
            Conv2DLayer(filters=filters, kernel_size=kernel_size, activation=ReLU(), padding="same")
        )

        if pooling_type == "max":
            cnn.add(MaxPooling2DLayer(pool_size=(2, 2)))
        else:
            cnn.add(AveragePooling2DLayer(pool_size=(2, 2)))

    # Add dense layers
    cnn.add(FlattenLayer())
    cnn.add(DenseLayer(input_dim=None, output_dim=64, activation=ReLU()))
    cnn.add(DenseLayer(input_dim=64, output_dim=10, activation=Softmax()))

    return cnn

def calculate_dense_input_dim(scratch_model, sample_input):
    # Forward pass through conv layers only
    output = sample_input

    conv_layer_count = 0
    for layer in scratch_model.layers:
        if isinstance(
            layer, (Conv2DLayer, MaxPooling2DLayer, AveragePooling2DLayer)
        ):
            output = layer.forward(output)
            conv_layer_count += 1
        elif isinstance(layer, FlattenLayer):
            output = layer.forward(output)
            # Update the first dense layer's input dimension
            for next_layer in scratch_model.layers[conv_layer_count + 1 :]:
                if (
                    isinstance(next_layer, DenseLayer)
                    and next_layer.input_dim is None
                ):
                    next_layer.input_dim = output.shape[1]
                    next_layer.weights = (
                        np.random.randn(next_layer.input_dim, next_layer.output_dim)
                        * 0.01
                    )
                    break
            break

In [6]:
# Create and train Keras model
keras_model = create_keras_model()
history = train_keras_model(keras_model)

# Evaluate Keras model
keras_pred = keras_model.predict(x_test[:1000])
keras_classes = np.argmax(keras_pred, axis=1)
keras_f1 = f1_score(y_test[:1000], keras_classes, average="macro")

# Create corresponding scratch model
scratch_model = create_scratch_model()

# Calculate dense layer input dimensions
sample_input = x_test[:1]
calculate_dense_input_dim(scratch_model, sample_input)

# Load weights from Keras to scratch model
scratch_model.load_weights_from_keras(keras_model)

# Test scratch model
scratch_pred = scratch_model.predict(x_test[:1000])
scratch_f1 = f1_score(y_test[:1000], scratch_pred, average="macro")

# Calculate agreement
keras_subset = np.argmax(keras_model.predict(x_test[:1000]), axis=1)
agreement = np.mean(keras_subset == scratch_pred) * 100

print(f"Keras F1-Score: {keras_f1:.4f}")
print(f"Scratch F1-Score: {scratch_f1:.4f}")
print(f"Agreement: {agreement:.1f}%")

/home/sandwicheese/Projects/Tubes2_ML/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748421034.756458   45142 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1748421037.062598   45283 service.cc:152] XLA service 0x7f8b20015c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748421037.062646   45283 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-05-28 15:30:37.087358: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748421037.267217   45283 cuda_dnn.cc:529] Loaded cuDNN version 90300


  43/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1365 - loss: 2.2782

I0000 00:00:1748421039.436638   45283 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.3617 - loss: 1.7297 - val_accuracy: 0.5715 - val_loss: 1.2100
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6068 - loss: 1.1152 - val_accuracy: 0.6395 - val_loss: 1.0126
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6899 - loss: 0.8912 - val_accuracy: 0.6946 - val_loss: 0.8789
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7302 - loss: 0.7772 - val_accuracy: 0.6993 - val_loss: 0.8612
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7622 - loss: 0.6763 - val_accuracy: 0.7268 - val_loss: 0.8019
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7948 - loss: 0.5911 - val_accuracy: 0.7352 - val_loss: 0.7999
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8197 - loss: 0.5148 - val_accuracy: 0.7351 - val_loss: 0.8018
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8388 - loss: 0.4561 - val_accurac